# Library of Composable Base Strategies
<https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html>

In [ ]:
pip install backtesting

In [2]:
from backtesting.test import SMA
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [4]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(GOOG, SmaCross, commission=.002)

stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   29.515829
Equity Final [$]                   31874.8637
Equity Peak [$]                   38202.59246
Return [%]                         218.748637
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   14.568116
Volatility (Ann.) [%]               18.649301
Sharpe Ratio                         0.781162
Sortino Ratio                        1.657039
Calmar Ratio                         0.623338
Max. Drawdown [%]                  -23.371122
Avg. Drawdown [%]                   -4.003572
Max. Drawdown Duration      674 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                   38
Win Rate [%]                        55.263158
Best Trade [%]                      48.410457
Worst Trade [%]                     -9.055262
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='5254', ...)